<a href="https://colab.research.google.com/github/EP111111/Doc_analysis/blob/main/DA_improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth as google_auth
google_auth.authenticate_user()


In [ ]:
!pip install shapely==1.8.5
!pip install google-cloud-aiplatform --upgrade
!pip install python-docx
!pip install langchain
!pip install docx2txt
!pip install chromadb

In [4]:
import vertexai
from vertexai.language_models import TextGenerationModel
from google.colab import drive
import docx
import numpy as np
import pandas as pd

In [ ]:
import time
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import Chroma
import pandas as pd
import json

In [5]:
PROJECT_ID = "ics-analysis-dev"
vertexai.init(project=PROJECT_ID, location="us-central1")

model = TextGenerationModel.from_pretrained("text-bison@001")

##Document Splitting
Document Splitting is to split documents into smaller chunks.

In [10]:
from langchain.document_loaders import Docx2txtLoader
text=Docx2txtLoader("Project Alpha.docx").load()

In [11]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

docs = text_splitter.split_documents(text)


##Vector Stores and Embeddings
 put these chunks into an index so that we are able to retrieve them easily when we want to answer questions on this document.

In [17]:
embeddings = VertexAIEmbeddings()
# Create the vector store
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="/content"
)

print(vectordb._collection.count())

47


In [20]:
from langchain.chat_models import ChatVertexAI
llm = ChatVertexAI()

In [21]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [47]:
question = """extract all questions in this document  :
"""
result = qa_chain({"query": question})
result["result"]

' 1. Describe the success of this experience and whether or how lessons learned were implemented.\n2. Describe your experience levels and PMP Certifications with project management. How do you ordinarily scope, estimate, assign resources and measure progress?\n3. Do you use internal staff for project management or do you use third party system integration teams?\n4. Which software development lifecycle model would you choose for this project and why?\n5. Describe your experience with Visual Fox Pro, any past data conversion experience with this technology, are any team members certified.\n6. Describe your testing methodology and QA approach. Which testing tools and instruments do you'

' ```json\n{\n  "questions": [\n    "Describe the success of this experience and whether or how lessons learned were implemented.",\n    "Describe your experience levels and PMP Certifications with project management. How do you ordinarily scope, estimate, assign resources and measure progress?",\n    "Do you use internal staff for project management or do you use third party system integration teams?",\n    "Which software development lifecycle model would you choose for this project and why?",\n    "Describe your experience with Visual Fox Pro, any past data conversion experience with this technology, are any team members certified.",\n    "Describe your testing methodology and'